<a href="https://colab.research.google.com/github/steimel60/ML/blob/main/DeepLearning/TabularData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
!pip install kaggle

     |████████████████████████████████| 719 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 38.6 MB/s 
     |████████████████████████████████| 4.2 MB 44.1 MB/s 
     |████████████████████████████████| 197 kB 67.0 MB/s 
     |████████████████████████████████| 346 kB 66.1 MB/s 
     |████████████████████████████████| 59 kB 8.6 MB/s 
     |████████████████████████████████| 212 kB 72.9 MB/s 
     |████████████████████████████████| 140 kB 73.7 MB/s 
     |████████████████████████████████| 1.1 MB 53.6 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
     |████████████████████████████████| 271 kB 71.6 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 67.0 MB/s 
     |████████████████████████████████| 112 kB 63.0 MB/s 
     |█████████████████

In [2]:
creds = '{"username":"dylansteimel","key":"b866f4bd21f4258d1089a99637b0af63"}'
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  cred_path.parent.mkdir(exist_ok=True)
  cred_path.mk_write(creds)
  cred_path.chmod(0o600)

import kaggle
api = kaggle.api

In [3]:
def file_extract(fname, dest=None):
     "Extract `fname` to `dest` using `tarfile` or `zipfile`."
     if dest is None: dest = Path(fname).parent
     fname = str(fname)
     if   fname.endswith('gz'):  tarfile.open(fname, 'r:gz').extractall(dest)
     elif fname.endswith('zip'): zipfile.ZipFile(fname     ).extractall(dest)
     else: raise Exception(f'Unrecognized archive: {fname}')

In [4]:
path = Path('/content/bluebook')
if not path.exists():
  path.mkdir()
  kaggle.api.competition_download_cli('bluebook-for-bulldozers', path=path)
file_extract(path/'bluebook-for-bulldozers.zip')
path.ls(file_type='text')

100%|██████████| 48.4M/48.4M [00:02<00:00, 23.3MB/s]


(#7) [Path('/content/bluebook/Valid.csv'),Path('/content/bluebook/median_benchmark.csv'),Path('/content/bluebook/TrainAndValid.csv'),Path('/content/bluebook/ValidSolution.csv'),Path('/content/bluebook/Test.csv'),Path('/content/bluebook/Machine_Appendix.csv'),Path('/content/bluebook/random_forest_benchmark_test.csv')]

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [7]:
#View sizes
df['ProductSize'].unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large', 'Compact'], dtype=object)

In [8]:
#Order the products by size
sizes = 'Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact'
df['ProductSize'] = df['ProductSize'].astype('category')
df['ProductSize'].cat.set_categories(sizes, ordered=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [9]:
#Kaggle tells us to us Root Mean Squared Log Error as our metric
#So we change our dependant variable - Sales Price - to it's log
dv = 'SalePrice'
df[dv] = np.log(df[dv])

This Section goes over Decision Trees which I go over in more detail in my ML/BaseReferences directory.